In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import requests
from bs4 import BeautifulSoup
import csv
import winsound as ws

def getStockCode(market):
    if market == 'kosdaq':
        url_market = 'kosdaqMkt'
    elif market == 'kospi':
        url_market = 'stockMkt'
    else:
        print('invalid market ')
        return
    url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13&marketType=%s' % url_market
    df = pd.read_html(url, header=0)[0]
    return df
if __name__ == '__main__':
    result_df = getStockCode('kospi')

In [57]:
def get_time_series(company_code):
    url = "https://finance.naver.com/item/frgn.nhn?code=" + company_code
    result = requests.get(url)
    ts_obj = BeautifulSoup(result.content,"html.parser")
    return ts_obj
    
def get_time_series2(company_code,j):
    url = "https://finance.naver.com/item/frgn.nhn?code=" + company_code + "&page=" + j
    result = requests.get(url)
    ts_obj = BeautifulSoup(result.content,"html.parser")
    return ts_obj

def load_series(company_code,i,ts_obj):
    temp=[]    
    
    ts_obj_table = ts_obj.findAll("table")
    ts_table = ts_obj_table[2]
    
    ts_tr = ts_table.findAll("tr")
    ts_tr_td = ts_tr[2+i].findAll("td")
    
    if(ts_tr_td[0].text!=''):
        temp.append(ts_tr_td[0].text.strip())
        temp.append(ts_tr_td[1].text.strip())
        temp.append(ts_tr_td[3].text.strip())
        temp.append(ts_tr_td[4].text.strip())
        temp.append(ts_tr_td[5].text.strip())
        temp.append(ts_tr_td[6].text.strip())
        temp.append(ts_tr_td[8].text.strip())
    return temp

# Main
#company_codes = ["011070","051900","032640"]
k=["2","3","4","5","6","7","8","9","10","11"]
company_codes = "207940"#종목명 입력
candle_chart_dataframe=[]
#for item in codsY['종목코드'][28:30]:
#for item in company_codes:
ts_obj = get_time_series(company_codes)
for i in range(1,31): 
    candle_chart_data = load_series(item,i,ts_obj)
    if candle_chart_data!=[]:
        candle_chart_dataframe.append(candle_chart_data)
for j in  k:
    ts_obj = get_time_series2(company_codes,j)
    for i in range(1,31): 
        candle_chart_data = load_series(item,i,ts_obj)
        if candle_chart_data!=[]:
            candle_chart_dataframe.append(candle_chart_data)
#print(candle_chart_dataframe)

# CSV 파일 출력
with open('200908_naver_time_sequence.csv','w',newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['날짜','종가','등락','거래량','기관','외국인','외국인보유율'])
    writer.writerows(candle_chart_dataframe)